# Samudra and SamudraAdjoint Demo
 
This notebook demonstrates how to use the Samudra ocean emulator model and its adjoint version for sensitivity analysis.

Samudra is the U-Net model designed by Dheeshjith et al. in "Samudra: An AI Global Ocean Emulator for Climate" (https://arxiv.org/abs/2412.03795). 

SamudraAdjoint is a subclass designed by Shaunticlair Ruiz (author of this document) to compute Adjoints from the Samudra model.
